<a href="https://colab.research.google.com/github/himanshisyadav/Webscraper_NER/blob/main/Markup_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.2 MB/s eta 0:00:00

In [2]:
from transformers import AutoProcessor, AutoModelForTokenClassification, get_scheduler, MarkupLMProcessor
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import evaluate
import pdb
import glob

In [3]:
!pip install optimum
from optimum.bettertransformer import BetterTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.7/243.7 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00
  Created wheel for optimum: filename=optimum-1.8.3-py3-none-any.whl size=318069 sha256=f7d52a9af1875998ae7dedaa91aeb6f2c39017bcca140236224ae23f7480fc69
  Stored in directory: /root/.cache/pip/wheels/12/a1/9c/a1637559c56182f7a32d41c1a98493fdfd8c765b88874611d8
Successfully built optimum
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.

In [4]:
class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=512):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx]
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, padding="max_length", max_length=self.max_length, return_tensors="pt", truncation=True)

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding


In [5]:
def get_labels(label_list, predictions, references):
    # Transform predictions and references tensos to numpy arrays

    y_pred = predictions.detach().clone().numpy()
    y_true = references.detach().clone().numpy()


    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return  true_predictions, true_labels

In [6]:
def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [7]:
from google.colab import drive
!fusermount -u gdrive
drive.mount("/content/gdrive",force_remount=True)
!ls
%cd gdrive/My Drive/project_folder

fusermount: failed to unmount /content/gdrive: No such file or directory
Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/project_folder


In [9]:
!ssh-keygen -t rsa -b 4096

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:wSi73Q0RsdP+oZCPCF9fC1dS3RRwUSxwH/MufeVzBD4 root@383b487ca14a
The key's randomart image is:
+---[RSA 4096]----+
|        o.  .o=XB|
|       o +   +ooB|
|    . . * . . E.=|
|     o   *   o *.|
|    o   S o + ..*|
|     = + B * o .+|
|    . + o = o    |
|                 |
|                 |
+----[SHA256]-----+


In [10]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-4b034cc8


In [11]:
!cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC1PTHz0ZF9ow5a3v0Uycksd7BvZrfRmc8R2DaVSpVCf3oXBFMfDE9jfQ+9fkS7wHhH/v6Hd6xM7cINcBhlSqkTMprEcH7kyskreTJmcXvFw7ToiUBsBhZB3V45NtG7thfI8Yz3mgXsLFcgyP4UiYBnajR/y1J2LAx7xF8O8qiEHBXv4xsdxYThYX9dDbed13osKW9mlJODiyVW35kG6F+jL0hlk6xfqoyaU9G8TlS1g8EkOuaoXraOv1sIq3vhBNZqdiXgciP5UZu+Djzv09W1v/NjoG/kaDtTYgF7zDrdvIi5u+RfTjP88S84XwxJYrFGdJPKgCkGSEBxPFr1mSo5kHsR+DlCd5oaePFbG5rKezXGFqx5SoRA7xZGW+VuWxtB+v+xPkh1jH6lFC6mMBNKc7nmYq5LfNqyMslm9J+/YT7drPXEkfkKQYmcxrS0m8EHD8VAN8ul5GR9ux1VmrhhzTS44nPTPN3Eqf0YPXj/mFVThkuNwN7gYfenNP0qsrDCun57D69uynhOwT1l3biLNGX2v9U17iENyIMDFCQDH4KmKHQpM6i7qoQlYNL9YRHpaPsxZvu9oz4HQ7ffWsqjMVU5afu4XlOJTL2i4x66M8MRTTqWw5otkWcVFVfaSjfNK1MEiNyFqc2VahFsdD9y97aagA4znTh9Sq+bF2P+mQ== root@383b487ca14a


In [12]:
!ssh -T git@github.com

git@github.com: Permission denied (publickey).


In [13]:
# ! git clone git@github.com:himanshisyadav/Webscraper_NER.git
! cd /content/gdrive/MyDrive/project_folder/Webscraper_NER/training_data/nodes_xpaths/

In [14]:
! pwd

/content/gdrive/MyDrive/project_folder


In [15]:
import glob
#Get data from directory ../training_data
files = glob.glob("Webscraper_NER/training_data/nodes_xpaths/Labeled_*.csv")

id2label = {0:'Address', 1:'Beds', 2:'Contact', 3:'NE', 4: 'Name', 5:'Price', 6:'Amenities'}
label2id = {label:id for id, label in id2label.items()}

data = []

for file in files:
    nodes_xpaths = pd.read_csv(file,delimiter=",")
    nodes_xpaths = nodes_xpaths.dropna()
    nodes_xpaths['Labels'].replace(['Address', 'Beds', 'Contact', 'NE', 'Name', 'Price', 'Amenities'], [0, 1, 2, 3, 4, 5, 6], inplace=True)
    nodes_xpaths = nodes_xpaths.rename({"nodes": "nodes", "Labels": "node_labels", "xpaths": "xpaths"}, axis = 'columns')
    cols = nodes_xpaths.columns.tolist()
    cols = [cols[0], cols[2], cols[1]]
    nodes_xpaths = nodes_xpaths[cols]
    data.append({'nodes': nodes_xpaths['nodes'].values.tolist(), 'xpaths': nodes_xpaths['xpaths'].values.tolist(), 'node_labels': nodes_xpaths['node_labels'].values.tolist()})

print(data)

[{'nodes': ['Apartments for Rent in Chicago, IL | Apartments.com', 'Menu', 'Sign In', 'Add a Listing', 'Back To Search', 'Min Rent', '$400', '$500', '$600', '$700', '$800', '$900', '$1,000', '$1,100', '$1,200', '$1,300', '$1,400', '$1,500', '$1,600', '$1,700', '$1,800', '$1,900', '$2,000', '$2,100', '$2,200', '$2,300', '$2,400', '$2,500', '$2,600', '$2,700', '$2,800', '$2,900', '$3,000', '$3,100', '$3,200', '$3,300', '$3,400', '$3,500', '$3,600', '$3,700', '$3,800', '$3,900', '$4,000', '$4,200', '$4,400', '$4,600', '$4,800', '$5,000', '$7,500', '$10,000', '$15,000', 'Max Rent', '$400', '$500', '$600', '$700', '$800', '$900', '$1,000', '$1,100', '$1,200', '$1,300', '$1,400', '$1,500', '$1,600', '$1,700', '$1,800', '$1,900', '$2,000', '$2,100', '$2,200', '$2,300', '$2,400', '$2,500', '$2,600', '$2,700', '$2,800', '$2,900', '$3,000', '$3,100', '$3,200', '$3,300', '$3,400', '$3,500', '$3,600', '$3,700', '$3,800', '$3,900', '$4,000', '$4,200', '$4,400', '$4,600', '$4,800', '$5,000', '$7,500

In [59]:
import glob
#Get data from directory ../training_data
files = glob.glob("Webscraper_NER/training_data/nodes_xpaths/Labeled_*.csv")

id2label = {0:'Address', 1:'Beds', 2:'Contact', 3:'NE', 4: 'Name', 5:'Price', 6:'Amenities'}
label2id = {label:id for id, label in id2label.items()}

data = []
nodes = []
xpaths = []
node_labels = []
for file in files:
    nodes_xpaths = pd.read_csv(file,delimiter=",")
    nodes_xpaths = nodes_xpaths.dropna()
    nodes_xpaths['Labels'].replace(['Address', 'Beds', 'Contact', 'NE', 'Name', 'Price', 'Amenities'], [0, 1, 2, 3, 4, 5, 6], inplace=True)
    nodes_xpaths = nodes_xpaths.rename({"nodes": "nodes", "Labels": "node_labels", "xpaths": "xpaths"}, axis = 'columns')
    cols = nodes_xpaths.columns.tolist()
    cols = [cols[0], cols[2], cols[1]]
    nodes_xpaths = nodes_xpaths[cols]
    nodes.append(nodes_xpaths['nodes'].values.tolist())
    xpaths.append(nodes_xpaths['xpaths'].values.tolist())
    node_labels.append(nodes_xpaths['node_labels'].values.tolist())
    # data.append({'nodes': nodes_xpaths['nodes'].values.tolist(), 'xpaths': nodes_xpaths['xpaths'].values.tolist(), 'node_labels': nodes_xpaths['node_labels'].values.tolist()})

nodes = [item for sublist in nodes for item in sublist]
xpaths = [item for sublist in xpaths for item in sublist]
node_labels = [item for sublist in node_labels for item in sublist]
data = [{'nodes': nodes, 'xpaths': xpaths, 'node_labels': node_labels} ]

In [60]:
print(data)

[{'nodes': ['Apartments for Rent in Chicago, IL | Apartments.com', 'Menu', 'Sign In', 'Add a Listing', 'Back To Search', 'Min Rent', '$400', '$500', '$600', '$700', '$800', '$900', '$1,000', '$1,100', '$1,200', '$1,300', '$1,400', '$1,500', '$1,600', '$1,700', '$1,800', '$1,900', '$2,000', '$2,100', '$2,200', '$2,300', '$2,400', '$2,500', '$2,600', '$2,700', '$2,800', '$2,900', '$3,000', '$3,100', '$3,200', '$3,300', '$3,400', '$3,500', '$3,600', '$3,700', '$3,800', '$3,900', '$4,000', '$4,200', '$4,400', '$4,600', '$4,800', '$5,000', '$7,500', '$10,000', '$15,000', 'Max Rent', '$400', '$500', '$600', '$700', '$800', '$900', '$1,000', '$1,100', '$1,200', '$1,300', '$1,400', '$1,500', '$1,600', '$1,700', '$1,800', '$1,900', '$2,000', '$2,100', '$2,200', '$2,300', '$2,400', '$2,500', '$2,600', '$2,700', '$2,800', '$2,900', '$3,000', '$3,100', '$3,200', '$3,300', '$3,400', '$3,500', '$3,600', '$3,700', '$3,800', '$3,900', '$4,000', '$4,200', '$4,400', '$4,600', '$4,800', '$5,000', '$7,500

In [16]:
! set CUDA_LAUNCH_BLOCKING=1

In [17]:
! pip install sadice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch

In [18]:
! pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=f5f989c38101b5e0a2f7e2b710a85878881b1ed9f80964b59ae6bfc21bf4d55f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [44]:
all_labels = []
for i, dat in enumerate(data):
  all_labels.append(dat['node_labels'])

all_labels = [item for sublist in all_labels for item in sublist]

label_ids = list(range(0,7))
counts = [all_labels.count(i) for i in label_ids]
print(label_ids)
print(counts)
class_weights = [sum(counts)/c for c in counts]
print(class_weights)

[0, 1, 2, 3, 4, 5, 6]
[149, 100, 100, 3190, 96, 100, 328]
[27.268456375838927, 40.63, 40.63, 1.273667711598746, 42.322916666666664, 40.63, 12.387195121951219]


In [62]:
from sadice import SelfAdjDiceLoss
processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-large")
processor.parse_html = False

dataset = MarkupLMDataset(data=data, processor=processor, max_length=512)

for example in dataset:
    for k,v in example.items():
        print(k,v.shape)

dataloader = DataLoader(dataset, batch_size=10, shuffle=True)


model = AutoModelForTokenClassification.from_pretrained("microsoft/markuplm-base", num_labels=7)

# print(model)

label_list = ["B-" + x for x in list(id2label.values())]
print(label_list)

optimizer = AdamW(model.parameters(), lr=5e-5)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

model.train()

for epoch in range(10):  # loop over the dataset multiple times
    for batch in tqdm(dataloader):
        # get the inputs;
        inputs = {k:v for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)
        logits = outputs.logits
        # labels = batch["labels"]

        # print(logits.size())
        # print(labels.size())
        # labels = torch.nn.functional.one_hot(labels, num_classes=7)

        # loss = criterion(logits.float().squeeze(), labels.squeeze())
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        preds, refs = get_labels(label_list, predictions, labels)
        metric = evaluate.load("seqeval")
        metric.add_batch( predictions=preds, references=refs,)

    eval_metric = compute_metrics()
    print(f"Epoch {epoch}:", eval_metric)
    print(f"Epoch {epoch}:", loss.item)

input_ids torch.Size([512])
token_type_ids torch.Size([512])
attention_mask torch.Size([512])
xpath_tags_seq torch.Size([512, 50])
xpath_subs_seq torch.Size([512, 50])
labels torch.Size([512])


Some weights of the model checkpoint at microsoft/markuplm-base were not used when initializing MarkupLMForTokenClassification: ['markuplm.pooler.dense.weight', 'nrp_cls.LayerNorm.bias', 'nrp_cls.decoder.bias', 'nrp_cls.dense.weight', 'ptc_cls.weight', 'nrp_cls.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'nrp_cls.dense.bias', 'markuplm.pooler.dense.bias', 'cls.predictions.decoder.bias', 'nrp_cls.LayerNorm.weight', 'ptc_cls.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model that 

['B-Address', 'B-Beds', 'B-Contact', 'B-NE', 'B-Name', 'B-Price', 'B-Amenities']


  0%|          | 0/1 [00:00<?, ?it/s]

Loss: 1.64750337600708
Epoch 0: {'Address_precision': 0.0, 'Address_recall': 0.0, 'Address_f1': 0.0, 'Address_number': 9, 'Amenities_precision': 0.0, 'Amenities_recall': 0.0, 'Amenities_f1': 0.0, 'Amenities_number': 0, 'Beds_precision': 0.0, 'Beds_recall': 0.0, 'Beds_f1': 0.0, 'Beds_number': 2, 'Contact_precision': 0.0, 'Contact_recall': 0.0, 'Contact_f1': 0.0, 'Contact_number': 2, 'NE_precision': 0.8545454545454545, 'NE_recall': 0.7230769230769231, 'NE_f1': 0.7833333333333332, 'NE_number': 130, 'Name_precision': 0.0, 'Name_recall': 0.0, 'Name_f1': 0.0, 'Name_number': 3, 'Price_precision': 0.0, 'Price_recall': 0.0, 'Price_f1': 0.0, 'Price_number': 3, 'overall_precision': 0.6308724832214765, 'overall_recall': 0.6308724832214765, 'overall_f1': 0.6308724832214765, 'overall_accuracy': 0.6308724832214765}
Epoch 0: <built-in method item of Tensor object at 0x7f5b43482070>


...

Loss: 0.211382657289505
Epoch 7: {'Address_precision': 1.0, 'Address_recall': 0.7777777777777778, 'Address_f1': 0.8750000000000001, 'Address_number': 9, 'Beds_precision': 0.0, 'Beds_recall': 0.0, 'Beds_f1': 0.0, 'Beds_number': 2, 'Contact_precision': 0.0, 'Contact_recall': 0.0, 'Contact_f1': 0.0, 'Contact_number': 2, 'NE_precision': 0.9154929577464789, 'NE_recall': 1.0, 'NE_f1': 0.9558823529411764, 'NE_number': 130, 'Name_precision': 0.0, 'Name_recall': 0.0, 'Name_f1': 0.0, 'Name_number': 3, 'Price_precision': 0.0, 'Price_recall': 0.0, 'Price_f1': 0.0, 'Price_number': 3, 'overall_precision': 0.9194630872483222, 'overall_recall': 0.9194630872483222, 'overall_f1': 0.9194630872483222, 'overall_accuracy': 0.9194630872483222}
Epoch 7: <built-in method item of Tensor object at 0x7f5b4db23d30>


  0%|          | 0/1 [00:00<?, ?it/s]

Loss: 0.17534516751766205
Epoch 8: {'Address_precision': 0.6923076923076923, 'Address_recall': 1.0, 'Address_f1': 0.8181818181818181, 'Address_number': 9, 'Beds_precision': 0.0, 'Beds_recall': 0.0, 'Beds_f1': 0.0, 'Beds_number': 2, 'Contact_precision': 0.0, 'Contact_recall': 0.0, 'Contact_f1': 0.0, 'Contact_number': 2, 'NE_precision': 0.9558823529411765, 'NE_recall': 1.0, 'NE_f1': 0.9774436090225563, 'NE_number': 130, 'Name_precision': 0.0, 'Name_recall': 0.0, 'Name_f1': 0.0, 'Name_number': 3, 'Price_precision': 0.0, 'Price_recall': 0.0, 'Price_f1': 0.0, 'Price_number': 3, 'overall_precision': 0.9328859060402684, 'overall_recall': 0.9328859060402684, 'overall_f1': 0.9328859060402684, 'overall_accuracy': 0.9328859060402684}
Epoch 8: <built-in method item of Tensor object at 0x7f5b4e123380>


  0%|          | 0/1 [00:00<?, ?it/s]

Loss: 0.15174643695354462
Epoch 9: {'Address_precision': 0.5625, 'Address_recall': 1.0, 'Address_f1': 0.72, 'Address_number': 9, 'Beds_precision': 0.0, 'Beds_recall': 0.0, 'Beds_f1': 0.0, 'Beds_number': 2, 'Contact_precision': 0.0, 'Contact_recall': 0.0, 'Contact_f1': 0.0, 'Contact_number': 2, 'NE_precision': 0.9774436090225563, 'NE_recall': 1.0, 'NE_f1': 0.988593155893536, 'NE_number': 130, 'Name_precision': 0.0, 'Name_recall': 0.0, 'Name_f1': 0.0, 'Name_number': 3, 'Price_precision': 0.0, 'Price_recall': 0.0, 'Price_f1': 0.0, 'Price_number': 3, 'overall_precision': 0.9328859060402684, 'overall_recall': 0.9328859060402684, 'overall_f1': 0.9328859060402684, 'overall_accuracy': 0.9328859060402684}
Epoch 9: <built-in method item of Tensor object at 0x7f5b4dca8630>


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
classes = ['Address', 'Beds', 'Contact', 'NE', 'Name', 'Price', 'Amenities']
cf_matrix = multilabel_confusion_matrix(refs, preds)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

In [77]:
html_string = """
<!DOCTYPE html>
<html>
<body>

<h1>801 S Plymouth Court</h1>
<h2>945-785-0099</h2>
<h3>This is heading 3</h3>
<h4>$900</h4>
<h5>Lame</h5>
<h6>Amli Loftsr</h6>

</body>
</html>

"""

from transformers import MarkupLMFeatureExtractor

processor.parse_html = True
encoding = processor(html_string, return_tensors="pt")

for k,v in encoding.items():
  print(k,v.shape)

import torch

# we use torch.no_grad() as we don't need any gradient computation here
# we're just doing inference! This saves memory
with torch.no_grad():
  outputs = model(**encoding)

logits = outputs.logits

print(logits.shape)







input_ids torch.Size([1, 26])
token_type_ids torch.Size([1, 26])
attention_mask torch.Size([1, 26])
xpath_tags_seq torch.Size([1, 26, 50])
xpath_subs_seq torch.Size([1, 26, 50])
torch.Size([1, 26, 7])
